In [ ]:
!pip uninstall openai -y
!pip install git+https://github.com/OpenGPTs-platform/openai-python

## Configure Client

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    base_url="http://localhost:8000",
    api_key="api_key",
)

In [ ]:
client.ops.web_retrieval.delete()

## Base Demo Without Tools

In [ ]:
assistant = client.beta.assistants.create(
    instructions="""Your job is to assist the assume the role of a news provider and inform the user of current news.
Always direct them to where they can learn more by providing the corresponding link.""",  # noqa
    name="News Provider Assistant",
    model="gpt-3.5-turbo",
)

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Show me interesting space themed news from ycombinator.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))

## Demo With `web_retireval` Tool

In [ ]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[
        {"type": "web_retrieval"}
    ],
)

In [ ]:
crawl = client.ops.web_retrieval.crawl_and_upsert(
    root_urls=["https://news.ycombinator.com/"],
    max_depth=1,
    description="Live news from ycombinator, a news feed centered on science and technology.",
    constrain_to_root_domain=False,
)
successful_crawls = [ci for ci in crawl.crawl_infos if ci.error is None]
print("Successful crawls: ", len(successful_crawls))
print(crawl.model_dump_json(indent=2))


In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Show me interesting space themed news from ycombinator.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))

## Demo With Tools `web_retireval` and `file_search`

In [ ]:
file = client.files.create(
    purpose="assistants",
    file=open("../assistants_api/assets/my_information.txt", "rb"),
)
vector_store = client.beta.vector_stores.create(
    name="Information About Me",
    file_ids=[file.id],
)
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    instructions="""Your job is to assist the assume the role of a news provider and inform the user of current news.
Always direct them to where they can learn more by providing the corresponding link.
You must begin by searching through the users files to find information about them.
Only then can you can look for relevant news.""",
    tools=[
        {"type": "web_retrieval"},
        {"type": "file_search"}
    ],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)
    

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Find me interesting news.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))